In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resnet-finetuned/model_resnet.pkl
/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:

test_path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip"

files = "/kaggle/working/"

import zipfile
    
with zipfile.ZipFile(test_path, 'r') as zipp:
    zipp.extractall(files)


test_dir = "/kaggle/working/test/"

filenames = os.listdir(test_dir)
labels = [x.split(".")[0] for x in filenames]

test_data = pd.DataFrame({"filename": filenames, "label": 'unknown'})

test_data.head()

,filename,label
0,8063.jpg,unknown
1,4335.jpg,unknown
2,8286.jpg,unknown
3,7896.jpg,unknown
4,55.jpg,unknown


In [3]:
import pickle


file_path = '/kaggle/input/resnet-finetuned/model_resnet.pkl'

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    model_resnet = pickle.load(file)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
NUM_CLASSES = 2
IMG_DIM = 224  
BATCH_SIZE = 100
CHANNEL_SIZE = 3

datagenerator = ImageDataGenerator( rescale=1./255)
                                    #rotation_range = 10,
                                    #horizontal_flip = True,
                                    #zoom_range = 0.2,
                                    #fill_mode = 'reflect',
                                    #width_shift_range = 0.1,
                                    #height_shift_range = 0.1)

test_datagenerator = datagenerator

test_generator = test_datagenerator.flow_from_dataframe(
    dataframe=test_data,
    directory='../working/test/',  # Directory containing the image files
    x_col='filename',   # Column with file names
    #y_col='label',      # Column with class labels
    class_mode=None,  # 
    batch_size=BATCH_SIZE,
    target_size=(IMG_DIM, IMG_DIM),
    shuffle=False
)

Found 12500 validated image filenames.


In [5]:
test_predict = model_resnet.predict(test_generator)

125/125 [==============================] - 48s 304ms/step


In [6]:
print(test_predict[:5])

[[1.0000000e+00 9.9429799e-16]
 [3.8847444e-27 1.0000000e+00]
 [4.2005628e-23 1.0000000e+00]
 [2.1546098e-13 1.0000000e+00]
 [1.0000000e+00 1.2536292e-32]]


In [7]:
y_test_pred = test_predict[:,1]
test_data['label'] = y_test_pred
submission_file = pd.DataFrame(columns=['Id', 'label'])
submission_file['Id']= test_data['filename'].apply(lambda x: int(x.split('.')[0]))
submission_file['label']= test_data['label']
submission_file.to_csv('submission.csv', index=False)
print(submission_file.head())

     Id         label
0  8063  9.942980e-16
1  4335  1.000000e+00
2  8286  1.000000e+00
3  7896  1.000000e+00
4    55  1.253629e-32
